In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../bench.py

In [4]:
data00_path = os.path.join('data00.json')
data01_path = os.path.join('data01.json')
data02_path = os.path.join('data02.json')
data10_path = os.path.join('data10.json')
data11_path = os.path.join('data11.json')
data12_path = os.path.join('data12.json')
data20_path = os.path.join('data20.json')
data21_path = os.path.join('data21.json')
data22_path = os.path.join('data22.json')

In [15]:
def mode_within(data_array, percent):
    max_sum_freq = 0
    mode = data_array[0]

    for check_val in set(data_array):
        sum_freq = sum(1 for ele in data_array if abs(ele - check_val) <= abs(percent * check_val / 100))

        if sum_freq > max_sum_freq:
            mode = check_val
            max_sum_freq = sum_freq

    return mode

In [16]:
# array = [23.2, 92.8, 91.0, 37.2, 82.0, 15.5, 79.3, 46.6, 98.1, 75.5, 78.9, 77.6,
#       33.8, 75.7, 96.8, 12.3, 18.4, 13.4, 6.0, 8.2, 25.8, 41.3, 68.5, 15.2,
#       74.7, 72.7, 18.0, 42.2, 36.1, 76.7, 1.2, 96.4, 4.9, 92.0, 12.8, 28.2,
#       61.8, 56.9, 44.3, 50.4, 81.6, 72.5, 12.9, 40.3, 12.8, 28.8, 36.3, 16.1,
#       68.4, 35.3, 79.2, 48.4, 97.1, 93.7, 77.0, 48.7, 93.7, 54.1, 65.4, 30.8,
#       34.4, 31.4, 78.7, 12.7, 90.7, 39.4, 86.0, 55.9, 6.8, 22.2, 65.3, 18.8,
#       7.1, 55.9, 38.6, 15.6, 59.2, 77.3, 76.9, 11.9, 19.9, 19.4, 54.3, 39.4,
#       4.0, 61.1, 16.8, 81.9, 49.3, 76.9, 19.2, 68.2, 54.4, 70.2, 89.8, 23.4,
#       67.5, 18.7, 10.8, 80.7, 80.3, 96.2, 62.3, 17.2, 23.0, 98.0, 19.1, 8.1,
#       36.2, 7.5, 55.9, 1.2, 56.8, 85.1, 18.9, 23.0, 13.5, 64.3, 9.1, 14.1, 14.1,
#       23.1, 73.2, 86.6, 39.1, 45.5, 85.0, 79.0, 15.8, 5.2, 81.5, 34.3, 24.3,
#       14.2, 84.6, 33.7, 86.3, 83.3, 62.8, 72.7, 14.7, 36.8, 92.5, 4.7, 30.0,
#       59.4, 57.6, 37.4, 22.0, 20.9, 61.6, 26.8, 47.1, 63.6, 6.0, 96.6, 61.2,
#       80.2, 59.3, 23.1, 29.3, 46.3, 89.2, 77.6, 83.2, 87.2, 63.2, 81.8, 55.0,
#       59.7, 57.8, 43.4, 92.4, 66.9, 82.1, 51.0, 22.1, 29.9, 41.0, 85.2, 61.5,
#       14.6, 48.0, 52.7, 31.4, 83.9, 35.5, 77.3, 35.8, 32.6, 22.2, 19.3, 49.1,
#       70.9, 43.9, 88.8, 56.3, 41.8, 90.3, 20.4, 80.4, 36.4, 91.5, 69.6, 75.3,
#       92.4, 84.8, 17.7, 2.3, 41.3, 91.3, 68.6, 73.3, 62.5, 60.5, 73.5, 70.7,
#       77.5, 76.8, 98.1, 40.9, 66.3, 8.6, 48.9, 75.4, 14.7, 35.9, 89.6, 15.1,
#       45.0, 77.6, 30.5, 76.1, 46.9, 34.3, 65.1, 43.9, 91.6, 88.8, 8.9, 42.9,
#       11.8, 32.1, 20.1, 48.9, 79.7, 15.3, 45.4, 80.1, 73.1, 76.5, 52.4, 9.6,
#       41.9, 52.7, 55.1, 30.9, 83.7, 46.7, 39.3, 40.5, 52.4, 19.2, 25.8, 52.7,
#       81.0, 38.0, 54.5, 15.3, 64.3, 88.3, 49.8, 90.5, 90.4, 79.7, 87.3, 32.3,
#       11.9, 5.7, 33.6, 75.1, 65.9, 29.1, 39.4, 87.5, 3.3, 66.3, 79.0, 97.9,
#       69.6, 22.0, 62.8, 97.1, 90.4, 39.5, 11.7, 30.3, 18.9, 34.6, 6.6]

# tensor = torch.tensor(array).reshape(1,-1,1)
# data = tensor.reshape(-1).numpy()
# print(data)
# print(mode_within(data,1))

In [17]:
def gen_param_func(data_tensor_array):
    # for 1 column, just have 1 element in tensor array
    data = data_tensor_array[0].reshape(-1).numpy()
    return [torch.tensor(mode_within(data,1))]

def model_func(param):
    class verifier_model(nn.Module):
        def __init__(self):
            super(verifier_model, self).__init__()
            self.w = nn.Parameter(data = param[0], requires_grad = False)

        def forward(self,X):
            # here is witness approach
            count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
            result = torch.zeros(X.size()[1])
            index_array = torch.tensor(range(X.size()[1]))

            for index in index_array:
                ele = X[0][index][0]
                if (torch.sum((torch.abs(X-ele)<=torch.abs(0.01*ele)).double())<=count_equal):
                    check_bool = torch.tensor(True)
                else:
                    check_bool = torch.tensor(False)
                result[index] = check_bool

            return (torch.sum(result) == X.size()[1], self.w)
    return verifier_model

In [19]:
bench_one([data00_path], model_func,gen_param_func, "Data00: 50 small values","default", "resources")

=====================================  Data00: 50 small values  =====================================


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:16: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:18: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":9,"param_scale":9,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":3936,"total_assignments":50,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,9],"model_input_scales":[9],"module_sizes":{"kzg":[],"poseidon":[3936,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.4.2","num_blinding_factors":null}


spawning module 0
spawning module 2
spawning module 0
spawning module 2
spawning module 0
spawning module 2


gen prf time:  0.5765950679779053
Theory result:  tensor(0.4600, dtype=torch.float64)
Our result:  [0.4609375]


In [20]:
bench_one([data01_path], model_func,gen_param_func, "Data01: 50 medium values","default", "resources")

=====================================  Data01: 50 medium values  =====================================


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:16: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:18: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":9,"param_scale":9,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":3936,"total_assignments":50,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,9],"model_input_scales":[9],"module_sizes":{"kzg":[],"poseidon":[3936,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.4.2","num_blinding_factors":null}


spawning module 0
spawning module 2
spawning module 0
spawning module 2
spawning module 0
spawning module 2


gen prf time:  0.5233378410339355
Theory result:  tensor(46.1000, dtype=torch.float64)
Our result:  [46.099609375]


In [21]:
bench_one([data02_path], model_func,gen_param_func, "Data02: 50 large values","default", "resources")

=====================================  Data02: 50 large values  =====================================


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:16: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:18: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":9,"param_scale":9,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":3936,"total_assignments":50,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,9],"model_input_scales":[9],"module_sizes":{"kzg":[],"poseidon":[3936,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.4.2","num_blinding_factors":null}


spawning module 0
spawning module 2
spawning module 0
spawning module 2
spawning module 0
spawning module 2


gen prf time:  0.5456831455230713
Theory result:  tensor(8594., dtype=torch.float64)
Our result:  [8594.0]


In [22]:
bench_one([data10_path], model_func,gen_param_func, "Data10: 300 small values","default", "resources")

=====================================  Data10: 300 small values  =====================================


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:16: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:18: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":9,"param_scale":9,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":300,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,9],"model_input_scales":[9],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.4.2","num_blinding_factors":null}


spawning module 0
spawning module 2
spawning module 0
spawning module 2
spawning module 0
spawning module 2


gen prf time:  1.7698299884796143
Theory result:  tensor(0.2300, dtype=torch.float64)
Our result:  [0.23046875]


In [23]:
bench_one([data11_path], model_func,gen_param_func, "Data11: 300 medium values","default", "resources")

=====================================  Data11: 300 medium values  =====================================


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:16: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:18: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":9,"param_scale":9,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":300,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,9],"model_input_scales":[9],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.4.2","num_blinding_factors":null}


spawning module 0
spawning module 2
spawning module 0
spawning module 2
spawning module 0
spawning module 2


gen prf time:  1.7967958450317383
Theory result:  tensor(77., dtype=torch.float64)
Our result:  [77.0]


In [24]:
bench_one([data12_path], model_func,gen_param_func, "Data12: 300 large values","default", "resources")

=====================================  Data12: 300 large values  =====================================


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:16: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:18: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":9,"param_scale":9,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":300,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,9],"model_input_scales":[9],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.4.2","num_blinding_factors":null}


spawning module 0
spawning module 2
spawning module 0
spawning module 2
spawning module 0
spawning module 2


gen prf time:  1.8070309162139893
Theory result:  tensor(9021., dtype=torch.float64)
Our result:  [9021.0]


In [28]:
bench_one([data20_path], model_func,gen_param_func, "Data20: 1000 small values",[4], "resources")

=====================================  Data20: 1000 small values  =====================================


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:16: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:18: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":4,"param_scale":4,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":45920,"total_assignments":1000,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,4],"model_input_scales":[4],"module_sizes":{"kzg":[],"poseidon":[45920,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.4.2","num_blinding_factors":null}


spawning module 0
spawning module 2
spawning module 0
spawning module 2
spawning module 0
spawning module 2


gen prf time:  5.746208906173706
Theory result:  tensor(0.6200, dtype=torch.float64)
Our result:  [0.625]


In [29]:
bench_one([data21_path], model_func,gen_param_func, "Data21: 1000 medium values",[3], "resources")

=====================================  Data21: 1000 medium values  =====================================


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:16: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:18: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":3,"param_scale":3,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":45920,"total_assignments":1000,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,3],"model_input_scales":[3],"module_sizes":{"kzg":[],"poseidon":[45920,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.4.2","num_blinding_factors":null}


spawning module 0
spawning module 2
spawning module 0
spawning module 2
spawning module 0
spawning module 2


gen prf time:  5.728955030441284
Theory result:  tensor(80.5000, dtype=torch.float64)
Our result:  [80.5]


In [30]:
bench_one([data22_path], model_func,gen_param_func, "Data22: 1000 large values",[2], "resources")

=====================================  Data22: 1000 large values  =====================================


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:16: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:18: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_79019/1129353987.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":45920,"total_assignments":1000,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[45920,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.4.2","num_blinding_factors":null}


spawning module 0
spawning module 2
spawning module 0
spawning module 2
spawning module 0
spawning module 2


gen prf time:  5.843122959136963
Theory result:  tensor(8540., dtype=torch.float64)
Our result:  [8540.0]
